In [1]:


%%capture
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
import os
import sys
from dotenv import load_dotenv
from pathlib import Path
import datetime as d 
load_dotenv()

# get the current directory
current_dir = os.getcwd()

# change the current working directory to the parent directory
os.chdir(os.path.dirname(current_dir))

path = os.getcwd()
parent = os.path.abspath(os.path.join(path, os.pardir))

USRPATH = "/Users/ducjeremyvu/dev/"


sys.path.append(Path().absolute().parent)
sys.path.append("/Users/ducjeremyvu/dev/wspace/nb")
sys.path.append(parent)
sys.path.append(...)

os.getcwd()


# from g_api import gservice


In [19]:
YEAR = 2024
MONTH = 11

In [2]:
import re
folder_path = "/Users/ducjeremyvu/Downloads/ucmdownload/PR_202411_135"

# Compile the regex pattern to find the substring between numbers_underscore and .csv
pattern = re.compile(r'\d+_\d+_(.*?)\.csv')


# Dictionary comprehension to extract the desired part from filename
folder_extract = {pattern.search(file.name).group(1): file 
                  for file in Path(folder_path).glob("*.csv") 
                  if pattern.search(file.name)}




# Dictionary comprehension to read CSV files into pandas DataFrames
csv_read = {key: pd.read_csv(path, sep=";", dtype=str)
            for key, path in folder_extract.items()}


In [3]:
data_wd =csv_read.get("wd")

data_wd["salary"] = data_wd["VD_EBETRAG"].str.replace(",",".").astype(float)

data_amounts_by_sequence = data_wd.groupby(["PNR","VERTNR"])["salary"].sum().reset_index()

data_amounts = data_amounts_by_sequence.groupby("PNR").agg(salary_sum=("salary","sum"),count_vertnr=("VERTNR","count"), salary_max=("salary","max")).reset_index()
data_amounts["user_id"] = data_amounts["PNR"].str[1:].astype(int)
data_amounts

,PNR,salary_sum,count_vertnr,salary_max,user_id
0,A07703,336.00,2,224.00,7703
1,A10164,1387.20,3,567.00,10164
2,A10189,178.50,1,178.50,10189
3,A10244,151.50,2,81.50,10244
4,A12084,203.00,1,203.00,12084
...,...,...,...,...,...
685,A89341,154.00,1,154.00,89341
686,A89365,113.75,1,113.75,89365
687,A89427,224.00,2,112.00,89427
688,A89433,200.00,2,100.00,89433


In [4]:

data_amounts_by_sequence["user_id"] = data_amounts_by_sequence["PNR"].str[1:].astype(int)

data_amounts_by_sequence

,PNR,VERTNR,salary,user_id
0,A07703,134,224.00,7703
1,A07703,135,112.00,7703
2,A10164,14,567.00,10164
3,A10164,15,357.60,10164
4,A10164,16,462.60,10164
...,...,...,...,...
928,A89427,1,112.00,89427
929,A89427,2,112.00,89427
930,A89433,1,100.00,89433
931,A89433,2,100.00,89433


In [5]:
path_fines = "/Users/ducjeremyvu/Downloads/ucmdownload/Fines_2024-12-09T17_32_16.csv"

data_fines = pd.read_csv(path_fines, sep=";")

data_fines

,user_id,amount,id,reason
0,72103,8.0,1927,Delay greater than First time per month 030 Min
1,73683,8.0,1928,Delay greater than First time per month 030 Min
2,60950,8.0,1934,Delay greater than First time per month 030 Min
3,75815,8.0,1940,Delay greater than First time per month 030 Min
4,76938,17.0,1942,Delay greater than First time per month 3160 Min
...,...,...,...,...
300,18420,150.0,3422,Noshow After job start No valid reason
301,18420,150.0,3423,Noshow After job start No valid reason
302,18420,150.0,3424,Noshow After job start No valid reason
303,86665,100.0,3425,Other


In [6]:
data_fines_merged = data_fines.merge(data_amounts, on="user_id",how="left")

data_fines_merged

,user_id,amount,id,reason,PNR,salary_sum,count_vertnr,salary_max
0,72103,8.0,1927,Delay greater than First time per month 030 Min,NaN,NaN,NaN,NaN
1,73683,8.0,1928,Delay greater than First time per month 030 Min,NaN,NaN,NaN,NaN
2,60950,8.0,1934,Delay greater than First time per month 030 Min,A60950,142.50,1.0,142.50
3,75815,8.0,1940,Delay greater than First time per month 030 Min,A75815,307.63,1.0,307.63
4,76938,17.0,1942,Delay greater than First time per month 3160 Min,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
300,18420,150.0,3422,Noshow After job start No valid reason,NaN,NaN,NaN,NaN
301,18420,150.0,3423,Noshow After job start No valid reason,NaN,NaN,NaN,NaN
302,18420,150.0,3424,Noshow After job start No valid reason,NaN,NaN,NaN,NaN
303,86665,100.0,3425,Other,A86665,178.50,1.0,178.50


In [7]:
data_fines_merged_only_valid = data_fines_merged.dropna()

data_fines_merged_only_valid

,user_id,amount,id,reason,PNR,salary_sum,count_vertnr,salary_max
2,60950,8.0,1934,Delay greater than First time per month 030 Min,A60950,142.50,1.0,142.50
3,75815,8.0,1940,Delay greater than First time per month 030 Min,A75815,307.63,1.0,307.63
8,61235,8.0,1960,Delay greater than First time per month 030 Min,A61235,1312.00,1.0,1312.00
10,76799,17.0,1971,Delay greater than First time per month 3160 Min,A76799,81.00,1.0,81.00
13,78246,8.0,1987,Delay greater than First time per month 030 Min,A78246,843.39,1.0,843.39
...,...,...,...,...,...,...,...,...
295,75358,150.0,3414,Noshow After job start No valid reason,A75358,139.86,1.0,139.86
296,75815,150.0,3416,Noshow After job start No valid reason,A75815,307.63,1.0,307.63
297,61970,50.0,3418,User Cancellation Sickness 01h Attest,A61970,276.00,1.0,276.00
303,86665,100.0,3425,Other,A86665,178.50,1.0,178.50


In [8]:
data_fines_merged_open = data_fines_merged[data_fines_merged["PNR"].isna()][["user_id","amount"]].rename(columns={"amount":"remaining_fine"})

data_fines_merged_open

,user_id,remaining_fine
0,72103,8.0
1,73683,8.0
4,76938,17.0
5,78158,8.0
6,78062,8.0
...,...,...
298,84699,150.0
299,84699,150.0
300,18420,150.0
301,18420,150.0


In [9]:
data_amounts_by_sequence[data_amounts_by_sequence["user_id"]==72220]

,PNR,VERTNR,salary,user_id


In [10]:
def custom_aggregate(strings, prefix):
    # Prefix each string and then join with a comma
    return prefix +  ', '.join(f"{x}" for x in strings)


In [11]:
data_fines_merged_only_valid_grouped = data_fines_merged_only_valid.groupby("user_id").agg({"amount":"sum","reason":lambda x: custom_aggregate(x, 'Vertragsstrafe: ')}).reset_index()
data_fines_merged_only_valid_grouped

,user_id,amount,reason
0,27595,440.0,Vertragsstrafe: User Cancellation Non Sickness...
1,32071,8.0,Vertragsstrafe: Delay greater than First time ...
2,41412,67.0,Vertragsstrafe: Delay greater than First time ...
3,48105,50.0,Vertragsstrafe: Delay greater than First time ...
4,48704,16.0,Vertragsstrafe: Delay greater than First time ...
...,...,...,...
75,85872,8.0,Vertragsstrafe: Delay greater than First time ...
76,86585,8.0,Vertragsstrafe: Delay greater than First time ...
77,86665,100.0,Vertragsstrafe: Other
78,87068,100.0,Vertragsstrafe: User Cancellation Sickness Aft...


In [12]:
data_fines_with_salaries_by_sequence = data_fines_merged_only_valid_grouped.merge(data_amounts_by_sequence, on="user_id", how="inner")#.to_csv("/Users/ducjeremyvu/Downloads/ucmdownload/test_fines.csv", index=False)

data_fines_with_salaries_by_sequence

,user_id,amount,reason,PNR,VERTNR,salary
0,27595,440.0,Vertragsstrafe: User Cancellation Non Sickness...,A27595,17,306.18
1,32071,8.0,Vertragsstrafe: Delay greater than First time ...,A32071,3,522.75
2,41412,67.0,Vertragsstrafe: Delay greater than First time ...,A41412,44,518.00
3,41412,67.0,Vertragsstrafe: Delay greater than First time ...,A41412,45,196.00
4,48105,50.0,Vertragsstrafe: Delay greater than First time ...,A48105,32,980.48
...,...,...,...,...,...,...
95,86585,8.0,Vertragsstrafe: Delay greater than First time ...,A86585,6,719.25
96,86585,8.0,Vertragsstrafe: Delay greater than First time ...,A86585,7,276.50
97,86665,100.0,Vertragsstrafe: Other,A86665,4,178.50
98,87068,100.0,Vertragsstrafe: User Cancellation Sickness Aft...,A87068,3,120.75


In [13]:
df = data_fines_with_salaries_by_sequence

# Initialize columns for tracking deductions and remaining fines
df['deducted'] = 0.0

# Group by user_id to process each user's data
grouped = df.groupby('user_id')

# To store users with remaining fines
remaining_fines = []

# Function to apply the deduction logic
def apply_deductions(group):
    fine = group['amount'].iloc[0]  # All rows in a group have the same fine value
    for index, row in group.iterrows():
        if fine <= 0:
            break
        deduction = min(row['salary'], fine)
        df.at[index, 'deducted'] = deduction
        fine -= deduction

    if fine > 0:
        remaining_fines.append({'user_id': group['user_id'].iloc[0], 'remaining_fine': fine})

# Apply the logic to each group
grouped.apply(apply_deductions)

df["VD_EBETRAG"]= df["deducted"].round(2)

# Create a DataFrame for remaining fines
remaining_fines_df = pd.DataFrame(remaining_fines)

# Show results
df.head(), remaining_fines_df.head()


(   user_id  amount                                             reason     PNR  \
 0    27595   440.0  Vertragsstrafe: User Cancellation Non Sickness...  A27595   
 1    32071     8.0  Vertragsstrafe: Delay greater than First time ...  A32071   
 2    41412    67.0  Vertragsstrafe: Delay greater than First time ...  A41412   
 3    41412    67.0  Vertragsstrafe: Delay greater than First time ...  A41412   
 4    48105    50.0  Vertragsstrafe: Delay greater than First time ...  A48105   
 
   VERTNR  salary  deducted  VD_EBETRAG  
 0     17  306.18    306.18      306.18  
 1      3  522.75      8.00        8.00  
 2     44  518.00     67.00       67.00  
 3     45  196.00      0.00        0.00  
 4     32  980.48     50.00       50.00  ,
    user_id  remaining_fine
 0    27595          133.82
 1    59698           43.25
 2    61970           74.00
 3    75358           68.14
 4    83669          292.50)

In [14]:
data_fines_for_import_input = df[df["deducted"] != 0]

data_fines_for_import_input = data_fines_for_import_input.rename(columns={"fine_id":"VD_TXT","reason":"VD_TXT"})

data_fines_for_import_input

,user_id,amount,VD_TXT,PNR,VERTNR,salary,deducted,VD_EBETRAG
0,27595,440.0,Vertragsstrafe: User Cancellation Non Sickness...,A27595,17,306.18,306.18,306.18
1,32071,8.0,Vertragsstrafe: Delay greater than First time ...,A32071,3,522.75,8.00,8.00
2,41412,67.0,Vertragsstrafe: Delay greater than First time ...,A41412,44,518.00,67.00,67.00
4,48105,50.0,Vertragsstrafe: Delay greater than First time ...,A48105,32,980.48,50.00,50.00
5,48704,16.0,Vertragsstrafe: Delay greater than First time ...,A48704,24,1814.00,16.00,16.00
...,...,...,...,...,...,...,...,...
94,85872,8.0,Vertragsstrafe: Delay greater than First time ...,A85872,4,596.25,8.00,8.00
95,86585,8.0,Vertragsstrafe: Delay greater than First time ...,A86585,6,719.25,8.00,8.00
97,86665,100.0,Vertragsstrafe: Other,A86665,4,178.50,100.00,100.00
98,87068,100.0,Vertragsstrafe: User Cancellation Sickness Aft...,A87068,3,120.75,100.00,100.00


In [15]:
remaining_fines_df

,user_id,remaining_fine
0,27595,133.82
1,59698,43.25
2,61970,74.00
3,75358,68.14
4,83669,292.50


In [16]:
remaining_fines_all = pd.concat([remaining_fines_df,data_fines_merged_open])
remaining_fines_all

,user_id,remaining_fine
0,27595,133.82
1,59698,43.25
2,61970,74.00
3,75358,68.14
4,83669,292.50
...,...,...
298,84699,150.00
299,84699,150.00
300,18420,150.00
301,18420,150.00


In [17]:
wd_column_order = [
    "SATZART",
    "FUNKTION",
    "MAN",
    "AK",
    "PNR",
    "NAME",
    "VORNAME",
    "VERTNR",
    "LA",
    "VD_ETAGE",
    "VD_ESTD",
    "VD_EFAKT",
    "VD_EBETRAG",
    "KST",
    "KOSTART",    
    "VD_KTR",
    "VD_DAT",
    "VD_ZDAT",
    "VD_HER",
    "VD_HER_DATE",
    "VD_PROZ",
    "VD_KSTB",
    "VD_USERID",
    "VD_WERT",
    "KST2_MAN",
    "KST2_AK",
    "VD_KALK",
   "VD_TXT",
]

def fill_var(dataframe, YEAR, MONTH):
    MONTH = str(MONTH) if len(str(MONTH)) == 2 else "0" + str(MONTH)
    YEAR_SHORT = str(YEAR)[2:4]
    dataframe = dataframe.copy()
    dataframe["MAN"] = "10246"
    dataframe["SATZART"] = "[VARTAB]"
    dataframe["FUNKTION"] = "INSERT"
    dataframe["AK"] = "201"
    dataframe["LA"] = "112"
    dataframe["VD_HER"] = f"2F{YEAR_SHORT}{MONTH}"
    dataframe["VD_ZDAT"] = f"{YEAR}-{MONTH}-01"
    dataframe["VD_DAT"] = ""
    dataframe["VD_ETAGE"] = ""
    dataframe["VD_EBETRAG"] = dataframe["VD_EBETRAG"].astype(str).str.replace(".",",")
    dataframe["KST"] = ""
    dataframe["KOSTART"] = ""
    dataframe["VD_KTR"] = ""
    dataframe["VD_ESTD"] =    dataframe["VD_EFAKT"] = ""
    dataframe["NAME"] = ""
    dataframe["VORNAME"] = "" 
    dataframe["VD_HER_DATE"] = ""
    dataframe["VD_PROZ"] = ""
    dataframe["VD_KSTB"] = ""
    dataframe["VD_USERID"] = ""
    dataframe["VD_WERT"] = ""
    dataframe["KST2_MAN"] = ""
    dataframe["KST2_AK"] = ""
    dataframe["VD_KALK"] = ""
#    dataframe["VD_TXT"] = dataframe["text"]



    return dataframe[wd_column_order]

In [20]:
export = fill_var(data_fines_for_import_input,YEAR, MONTH)

file_name = f"{YEAR}_{MONTH}_fines_manual_student_service"

export.to_csv("/Users/ducjeremyvu/Downloads/ucmdownload/" + file_name + ".csv", index=False, sep=";")

file_name_remaining = f"{YEAR}_{MONTH}_fines_manual_student_service_remaining"

remaining_fines_all.to_excel("/Users/ducjeremyvu/Downloads/ucmdownload/" + file_name_remaining + ".xlsx", index=False)